In [ ]:
import pandas as pd
import urllib
import requests
import numpy as np
import os
import datetime
from bs4 import BeautifulSoup

# URLを取得し、解析する #
def get_and_bs_html(dic):
    html = requests.get(dic["URL"]+dic["Conditions"])
    if dic["Name"] == "zero-one":
        bs = BeautifulSoup(html.content,"html.parser")
    else:
        bs = BeautifulSoup(html.text,"html.parser")
    return bs

# 新着記事かどうかの判断基準を決める #
def select_criterion(dic,df):
    df_sorted_by_source = df[df["Source"]==dic["Name"]]
    if (dic["Name"] == "Infra") | (dic["Name"] == "Wantedly"):
        latest_date = list(df_sorted_by_source[-1:]["Date"].values)[0]
        criterion = df_sorted_by_source[df_sorted_by_source["Date"]==latest_date][:1]["Title"].values # latest title
    else:
        criterion = list(df_sorted_by_source["Title"]) # titles_stored
    return criterion, df_sorted_by_source

# サイトによって異なるスクレイピング関数 #
def search_new_article(dic,df):
    # 前準備 #
    columns = ["Title","URL","Summary","Source","Date"]
    today = datetime.date.today().strftime("%Y/%m/%d")
    bs = get_and_bs_html(dic)
    criterion,df_sorted_by_source = select_criterion(dic,df)
    # 本番 #
    if dic["Name"] == "キャリアバイト": # キャリアバイト用
        all_articles = bs.find_all("a",{"class":"tapArea informClick"})
        for article in all_articles:
            avalibility = article.find("span",{"class":"newLabel"})
            if avalibility == None: # Newのラベルがなければbreak
                break
            else:
                title = article.find("h3",{"class":"summaryList_ttl"})
                for e in title:
                    if (e.string != "NEW") & (len(e.string)>1):
                        title_ = e.string.strip("\n ") # タイトルを特定
                    else:
                        continue
                if title_ not in criterion: # 新着記事であれば
                    url = dic["URL"]+article.get("href") # URLを特定
                    # summaryを特定 #
                    summary = article.find("div",{"class":"summaryList_lead"}).string
                    print(title_)
                    print(url)
                    print(summary)
                    se = pd.Series([title,url,summary,"キャリアバイト",today],index=columns)
                    df = df.append(se,ignore_index=True)
                else:
                    continue
        # 新着記事がなければ #
        if df["Source"].value_counts()[dic["Name"]] == df_sorted_by_source["Source"].value_counts()[dic["Name"]]:
            print(dic["Name"]+"に新着求人はない")
            print(dic["URL"]+dic["Conditions"])
    if dic["Name"] == "Infra": # Infra用
        all_articles = bs.find_all("div",{"class":"col s12 m6"})
        for article in all_articles:
            title_and_url = article.find("h3",{"class":"intern-title"})
            title = title_and_url.find("a").string.strip("\n ") # タイトルを特定
            # 新着記事がなければ #
            if title == criterion:
                break
            # 新着記事であれば #
            else:
                # URLを取得 #
                url = dic["URL"]+title_and_url.find("a").get("href")
                # summaryを取得 #
                summary = article.find("td",{"class":"job-type"}).string
                print(title)
                print(url)
                print(summary)
                se = pd.Series([title,url,summary,"Infra",today],index=columns)
                df = df.append(se,ignore_index=True)
        # 新着記事がなければ #
        if df["Source"].value_counts()[dic["Name"]] == df_sorted_by_source["Source"].value_counts()[dic["Name"]]:
            print(dic["Name"]+"に新着求人はない")
            print(dic["URL"]+dic["Conditions"])
    if dic["Name"] == "zero-one": # ゼロワン用
        all_articles = bs.find_all("section",{"class":"l-common-joblistBox"})
        for article in all_articles:
            avalibility = article.find("div",{"class":"l-common-joblist_fav_sp display-none_pc"}).find("span").string
            if avalibility != "お気に入り": # 募集中でなければbreak
                break
            else:
                title_and_url = article.find("div",{"class":"l-common-joblist_text"})
                # タイトルを取得 #
                title = title_and_url.find("h4").string.strip("\n ")
                # 募集中でありかつ新着であれば #
                if title not in criterion:
                    # URLを取得 #
                    url = dic["URL"] + title_and_url.find("a").get("href")
                    summary_and_loc = article.find_all("dl",{"class":"display-none_sp"})
                    # summaryを取得 #
                    summary = summary_and_loc[1].find("dd").string.strip("\n ")
                    print(title)
                    print(url)
                    print(summary)
                    se = pd.Series([title,url,summary,"zero-one",today],index=columns)
                    df = df.append(se,ignore_index=True)
                else:
                    continue
        # 新着記事がなければ #
        if df["Source"].value_counts()[dic["Name"]] == df_sorted_by_source["Source"].value_counts()[dic["Name"]]:
            print(dic["Name"]+"に新着求人はない")
            print(dic["URL"]+dic["Conditions"])
    if dic["Name"] == "Wantedly": # Wantedly用
        all_articles = bs.find_all("article",{"class":"projects-index-single"})
        for article in all_articles:
            title = article.find("h1",{"class":"project-title"}).find("a").string # タイトルを特定
            # 新着記事がなければ #
            if title == criterion:
                break
            # 新着記事であれば #
            else:
                # URLを取得 #
                url = dic["URL"]+article.find("h1",{"class":"project-title"}).find("a").get("href")
                # summaryを取得 #
                summary = article.find("p",{"class":"project-excerpt"}).string.strip("\n ")
                print(title)
                print(url)
                print(summary)
                se = pd.Series([title,url,summary,"Wantedly",today],index=columns)
                df = df.append(se,ignore_index=True)
        # 新着記事がなければ #
        if df["Source"].value_counts()[dic["Name"]] == df_sorted_by_source["Source"].value_counts()[dic["Name"]]:
            print(dic["Name"]+"に新着求人はない")
            print(dic["URL"]+dic["Conditions"])
    return df
# サイトおよび条件の辞書 #
dic_career = {
    "Name":"キャリアバイト",
    "URL":"https://careerbaito.com",
    "Conditions":"/search/38/305/316"
}
dic_infra = {
    "Name":"Infra",
    "URL":"https://www.in-fra.jp/",
    "Conditions":"long-internships?occupation%5B%5D=1&occupation%5B%5D=7&area%5B%5D=1&area%5B%5D=4&area%5B%5D=5&area%5B%5D=2&industry%5B%5D=1&industry%5B%5D=4&industry%5B%5D=11&order=recent"
}
dic_zeroone = {
    "Name":"zero-one",
    "URL":"https://01intern.com",
    "Conditions":"/job/list.html?areas=13&jobTypes=6&jobTypes=2&businessTypes=1&businessTypes=4&businessTypes=8"
}
dic_wantedly = {
    "Name":"Wantedly",
    "URL":"https://www.wantedly.com/",
    "Conditions":"/projects?type=recent&page=1&occupation_types%5B%5D=jp__engineering&hiring_types%5B%5D=internship&fields%5B%5D=jp__data_scientist&locations%5B%5D=tokyo"
}
dic_list = [dic_career,dic_infra,dic_zeroone,dic_wantedly]

In [ ]:
csv_path = os.path.join("datasets","Internship.csv")
df = pd.read_csv(csv_path,index_col=0)
df

In [ ]:
for target in dic_list:
    df = search_new_article(target,df=df)

In [ ]:
df.to_csv(csv_path,encoding='utf_8_sig')

In [ ]:
df